In [2]:

import dill
import os
import numpy as np
import pandas as pd
import torch
from extract_summary_finetune.model import lstm,transformer


In [29]:
model_dict={
    'lstm1':lstm,
    'lstm2':lstm,
    'lstm3':lstm,
    'lstm4':lstm,   
    'transformer1':transformer,
    'transformer2':transformer,
    'transformer1':transformer,
    'transformer2':transformer 
}
model_setting_parameters = {
    'lstm1':{'input_size':1024, 'hidden_size':64, 'num_layers':1, 'dropout':0},
}
model_results_dict = {}
pred_model_dict = {}
model_list=['lstm1']
for network in model_list:
    file_name = f'summary_{network}.dil'
    load_path =  os.path.join("/home/yuxin.fan/result/",file_name)
    with open(load_path, "rb") as file:
        network_results_dict = dill.load(file)
    model_results_dict[network] = network_results_dict
    
    
    network_state_dict = network_results_dict['model_state_dict']
    pred_network = model_dict[network](**model_setting_parameters[network])
    pred_network.load_state_dict(network_state_dict, strict=True)
    

In [39]:
test_tensor1 = torch.tensor(np.load("/home/yuxin.fan/feature_from_swufe_bert.npy",allow_pickle=True)).type(torch.float32)

predict = pred_network(test_tensor1)
predict = predict.ge(0.2)
predict.type(torch.int)
from extract_summary_finetune.performance_measures import calc_auc_score
calc_auc_score(predict,predict)

1.0

In [36]:

content_data = []
with open('/home/yuxin.fan/short_news_text1.txt','r',encoding='utf-8') as f:
    for line in f:
        content_data.append(line)
content = content_data[0].strip('[]').strip('\'').split('\', \'')
content_filled = []
for news in content:
    #print(len(news))
    if len(news) < 512:
        news_filled = news+' '*(512 - len(news))
        content_filled.append(news_filled)
    else:
        news_filled = news[:512]
        content_filled.append(news_filled) 
    #print(len(news_filled)) 
content_word = [list(item) for item in content_filled]

print(predict)
summary = []
for i in range(predict.shape[0]):
    each_summary = ''
    for j in range(predict.shape[1]):
        each_summary= each_summary + (predict[i][j]*content_word[i][j])
    summary.append(each_summary)

print(summary[0])
print(content[0])


content_summary = pd.DataFrame.from_dict({'summary':summary,'content':content})
content_summary.to_csv("/home/yuxin.fan/content_summary.csv",index=False ,encoding='utf_8_sig')

content_summary.head(10)

tensor([[[ True],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        [[ True],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        [[ True],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        ...,

        [[ True],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        [[ True],
         [ True],
         [ True],
         ...,
         [False],
         [False],
         [False]],

        [[ True],
         [ True],
         [ True],
         ...,
         [ True],
         [ True],
         [ True]]])
[汽车之车官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万续航版的ND里6
[汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元。长续航版的NEDC续航里程为660km，0-

summary  \
0  [汽车之车官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Per...   
1  揭秘Siri，苹果发布论文阐释语音助手设计想法就多任务处理、多语言识别等问题，苹果在论文中给...   
2  在2018年的大牛行情过后，2019年的债市将如何演绎？多位基金经理在接受记者采访时表示，去...   
3  记者 ，总投资921亿元的广西“信息网”基础设施建设三年大会战正式启动。据了解，今年2月份，...   
4  从某种程度上来看，这是因为腾讯是一个强大的生态企业，而且有着诸多的现金奶牛业务，如我们熟悉的...   
5  陇西县焦家湾村建起了别具特色的民俗文化墙，激发了贫困群众脱贫信心。”从甘肃陇西县城坐车20多...   
6  近日，山东省文化和旅游厅印发《“好客服务”——全省旅游服务质量提升三年行动（2021-202...   
7  ”3月28日，位于孔子故里山东曲阜的孔子博物馆青少年活动中心，一场别开生面的“射柳”体验活动...   
8  日前，记者从格兰仕顺德总部获悉，基于工业4.0、开源芯片两大科技项目的成熟推进与阶段性进展，...   
9  “党建联盟工程”助力脱贫攻坚。今年以来，贺州市平桂区以“围绕一个目标、探索一种组织设置、搭建...   

                                             content  
0  [汽车之家 新车上市] 日前，我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参...  
1  揭秘Siri，苹果发布论文阐释语音助手设计想法就多任务处理、多语言识别等问题，苹果在论文中给...  
2  在2018年的大牛行情过后，2019年的债市将如何演绎？多位基金经理在接受记者采访时表示，去...  
3  记者周骁骏 童政报道：日前，总投资921亿元的广西“信息网”基础设施建设三年大会战正式启动。...  
4  从某种程度上来看，这是因为腾讯是一个强大的生态企业，而且有着诸多的现金奶牛业务，如我们熟悉的...  
5  陇西县焦家湾村建起了别具特色的民俗文化墙，激发了贫困群众脱贫信心。”从甘肃陇西县城坐车20多...  
6  近日，山东省文化和旅游厅印发《“好客服务”——全省旅游服务质量提升三年行动（2021-202...  
7  ”3月28日，位于孔子故里山东曲阜的孔子博物馆青少年活动中心，一场别开生面的“射柳”体验活动...  
8  日前，记者从格兰仕顺德总部获悉，基于工业4.0、开源芯片两大科技项目的成熟推进与阶段性进展，...  
9  “党建联盟工程”助力脱贫攻坚。今年以来，贺州市平桂区以“围绕一个目标、探索一种组织设置、搭建...

In [40]:
x = torch.randn((10,51,10))
x.ndim

3

In [41]:
import math
math.log([1,1,1,22,1,2,])

TypeError: must be real number, not list

In [51]:
a= [1,8]
for i in range(2):
    a[i] = 1
np.mean(a)

1.0

In [3]:
import torch.nn as nn



In [4]:
weight1 = torch.tensor([[[0.7000],
         [0.7000]],
        [[0.3000],
         [0.3000]]])

weight2= torch.tensor([[[1.3000],
         [0.7000]],
        [[1.3000],
         [0.7000]]])

In [5]:
inputs = torch.tensor([[[0.5000],
         [0.5000]],
        [[0.0000],
         [1.0000]]])

targets = torch.tensor([[[1.0000],
         [0.0000]],
        [[0.0000],
         [1.0000]]])

In [6]:
loss = nn.BCELoss()(inputs,targets.float())
loss

tensor(0.3466)

In [7]:
loss = nn.BCELoss(weight=weight1)(inputs,targets.float())
loss

tensor(0.2426)

In [8]:
loss = nn.BCELoss(weight=weight2)(inputs,targets.float())
loss


tensor(0.3466)

In [89]:
w= torch.tensor([[1,2],[3,4]],dtype=float)
w

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [10]:
from extract_summary_finetune.loss_weighted import loss_weighted
loss_weighted(inputs,targets)

tensor([[[1.0000],
         [1.0000]],

        [[0.7737],
         [1.2263]]], dtype=torch.float64)

In [11]:
loss = nn.BCELoss(weight=loss_weighted(inputs,targets).float())(inputs,targets.float())
loss

tensor(0.3466)